In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_rel, wilcoxon
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import numpy as np


ured_normalized = pd.read_csv('usa-real-estate-dataset-normalized.csv')

zip_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ured_normalized[['zip_code']] = zip_encoder.fit_transform(ured_normalized[['zip_code']])
ured_normalized['zip_code'] = ured_normalized['zip_code'].astype('category')

y = ured_normalized['price']
X_geo = ured_normalized.drop(columns=['price'])
X_no_geo = ured_normalized.drop(columns=['price','state', 'city'])

top_cities = X_geo['city'].value_counts().nlargest(200).index
X_geo['city'] = X_geo['city'].where(X_geo['city'].isin(top_cities), '__other__')

X_geo = pd.get_dummies(X_geo, columns=['city', 'state'], drop_first=True)


X_train_geo, X_test_geo, y_train, y_test = train_test_split(X_geo, y, test_size=0.2, random_state=2137)
X_train_no_geo, X_test_no_geo, _, _ = train_test_split(X_no_geo, y, test_size=0.2, random_state=2137)

model_geo = XGBRegressor()
model_geo.load_model("models/xgb_model_200c_50s_zipcode_cat.json")

model_no_geo = XGBRegressor()
model_no_geo.load_model("models/xgb_model_zipcode_cat.json")

preds_geo = model_geo.predict(X_test_geo)
preds_no_geo = model_no_geo.predict(X_test_no_geo)

mae_geo = mean_absolute_error(y_test, preds_geo)
mae_no_geo = mean_absolute_error(y_test, preds_no_geo)

print(f"MAE with geo features:     {mae_geo:.2f}")
print(f"MAE without geo features:  {mae_no_geo:.2f}")

errors_geo = np.abs(y_test - preds_geo)
errors_no_geo = np.abs(y_test - preds_no_geo)


t_stat, p_value_t = ttest_rel(errors_no_geo, errors_geo)
print(f"Paired t-test p-value: {p_value_t:.5f}")

try:
    stat_w, p_value_w = wilcoxon(errors_no_geo, errors_geo)
    print(f"Wilcoxon test p-value: {p_value_w:.5f}")
except ValueError as e:
    print(f"Wilcoxon test failed: {e}")


MAE with geo features:     174866.49
MAE without geo features:  200263.55
Paired t-test p-value: 0.00000
Wilcoxon test p-value: 0.00000


In [ ]:
t_stat, p_value_t = ttest_rel(errors_no_geo, errors_geo)
print(f"Paired t-test p-value: {p_value_t}")

try:
    stat_w, p_value_w = wilcoxon(errors_no_geo, errors_geo)
    print(f"Wilcoxon test p-value: {p_value_w}")
except ValueError as e:
    print(f"Wilcoxon test failed: {e}")


Paired t-test p-value: 0.0
Wilcoxon test p-value: 0.0
